# Update LAS File with Bad Projection VLRs
In addition to the lack of projection information in some LAS files (that was fixed in the proj_fix.ipynb)
some LAS tiles had total incorrect projection information. This utility updates the VLR information to
contain a projection information for a correct LAS tile.

__NOTE: This code is extremely specific to this problem but shows the approach should be adaptable.__

In [25]:
import os
import copy
import geopandas as gpd
from laspy.file import File

In [26]:
df = gpd.read_file('../lidar/cumberland/tiles.gpkg')

# read the first tile (assuming its good)
f = File(df.loc[0,'location'].replace('/data/isgs', '..'))
good_vlrs = good.header.vlrs
f.close()

bad_epsg = df[df['srs'] == 'EPSG:3444']
bad_epsg.head()

,location,srs,modified,created,geometry
2148,/data/isgs/lidar/cumberland/las/960_968.las,EPSG:3444,2013-05-08T19:20:06,2018-12-13T21:45:11,"POLYGON ((-94.8839925776402 39.22932909521189,..."
2545,/data/isgs/lidar/cumberland/las/958_968.las,EPSG:3444,2013-05-08T19:20:58,2018-12-13T21:45:13,POLYGON ((-94.89102894452813 39.22904327813905...


In [38]:
def fix(loc):
    path = os.path.join('../fix', os.path.basename(loc))
    os.system('cp {} {}'.format(loc.replace('/data/isgs','..'), path))
    
    bad = File(path, mode='rw')
    vlrs = copy.copy(bad.header.vlrs)
    vlrs[0] = good_vlrs[0]
    vlrs[1] = good_vlrs[1]
    vlrs[2] = good_vlrs[2]
    bad.header.vlrs = vlrs
    bad.close()
    return path

bad_epsg['location'].apply(fix)

2148    ../fix/960_968.las
2545    ../fix/958_968.las
Name: location, dtype: object